In [99]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nfl_data_py as nfl
import dataframe_image as dfi

In [165]:
##draft=pd.read_csv('..\\Data\\NFL-Draft-8515\\nfl_draft.csv')
players=pd.read_csv('..\\Data\\NFL-Stats-1222\\yearly.csv')
years=range(1991,2024)
draft=pd.DataFrame(nfl.import_draft_picks(years))

draft.to_csv('..\\Data\\NFL-Draft-9123.csv', index=False)



In [166]:
active_players=players[players['season']==2022]
retired_players=players[players['name'].isin(active_players['name'])==False]

In [167]:
rp=retired_players[['name','season']]
finalYear=rp.groupby('name').max('season').reset_index()
firstYear=rp.groupby('name').min('season').reset_index()
d=draft[draft['pfr_player_name'].isin(finalYear['name'])].sort_values('pfr_player_name').reset_index(drop=True)
d=d.rename(columns={'pfr_player_name':'name','season':'draft_year'})
d=d[['name','draft_year', 'round', 'pick', 'team']]

retired=d.merge((finalYear.merge(firstYear, on='name', how='left')), on='name', how='left')

In [173]:
retired['years_played']=retired['season_x']-retired['season_y']+1
averageByRound=retired.groupby('round').mean()
averageByRound=averageByRound.rename(columns={'years_played':'Mean Career Length'})

seCareer=retired.groupby('round').sem()
averageByRound['Standard Error']=seCareer['years_played']
averageByRound.drop(averageByRound.tail(1).index,inplace=True)
averageByRound.index.names=['Round']
display(averageByRound)
dfi.export(averageByRound[['Mean Career Length', 'Standard Error']].style.format("{:.3f}"), 'meanNFLCareer.png')


,draft_year,pick,season_x,season_y,Mean Career Length,Standard Error
Round,,,,,,
1,2007.846939,15.938776,2016.132653,2012.602041,4.530612,0.265961
2,2009.627660,48.946809,2016.191489,2012.946809,4.244681,0.240436
3,2010.730769,81.721154,2016.326923,2013.403846,3.923077,0.256248
4,2012.147541,116.950820,2016.467213,2013.975410,3.491803,0.207972
5,2013.318681,154.901099,2017.384615,2014.725275,3.659341,0.251112
6,2012.514286,191.504762,2016.838095,2014.400000,3.438095,0.232018
7,2012.224490,233.530612,2016.040816,2014.418367,2.622449,0.197563


In [154]:
salaries=pd.read_csv('..\\Data\\nfl_salaries.csv')
salaries=salaries.rename(columns={'Player':'name'})
playerSalaries=players.merge(salaries, on='name', how='left')
playerSalaries=playerSalaries.merge(d, on='name', how='left')


In [155]:
playerSalaries['Salary'] = playerSalaries['Salary'].replace('[\$,]', '', regex=True).astype(float)


In [163]:
averageByRound=playerSalaries.groupby('round_y').mean()
averageByRound.style.format("{:.3f}")
averageByRound.index.names = ['Round']
averageByRound=averageByRound.rename(columns={'Salary':'Mean Salary'})
averageByRound=averageByRound.dropna()
seSal=playerSalaries.groupby('round_y').sem()
averageByRound['Standard Error']=seSal['Salary']

averageByRound.index.names = ['Round']
averageByRound.index=averageByRound.index.astype(int)

dfi.export(averageByRound[['Mean Salary', 'Standard Error']].style.format("{:.0f}"), 'meanNFLSalary.png')


## NHL Stats

In [157]:
nhl_draft=pd.read_csv('..\\Data\\NHL-Draft\\nhldraft.csv')
display(nhl_draft.columns)
nhl_draft=nhl_draft[nhl_draft['year']>1990]
teams={}
for i in range(1991,2023):
    teams[i]=(len(nhl_draft[nhl_draft['year']==i]['team'].unique()))
round=[]
for index, row in nhl_draft.iterrows():
    round.append((row['overall_pick']//teams[row['year']])+1)
nhl_draft['round']=round

skaters=pd.read_csv('..\\Data\\NHL-Stats-0922\\skaters.csv')
goalies=pd.read_csv('..\\Data\\NHL-Stats-0922\\goalies.csv')
gl_draft=nhl_draft[nhl_draft['position']=='G']
sk_draft=nhl_draft[nhl_draft['position']!='G']
display(skaters.columns)

Index(['id', 'year', 'overall_pick', 'team', 'player', 'nationality',
       'position', 'age', 'to_year', 'amateur_team', 'games_played', 'goals',
       'assists', 'points', 'plus_minus', 'penalties_minutes',
       'goalie_games_played', 'goalie_wins', 'goalie_losses',
       'goalie_ties_overtime', 'save_percentage', 'goals_against_average',
       'point_shares'],
      dtype='object')

Index(['playerId', 'season', 'name', 'team', 'position', 'situation',
       'games_played', 'icetime', 'shifts', 'gameScore',
       ...
       'OffIce_F_xGoals', 'OffIce_A_xGoals', 'OffIce_F_shotAttempts',
       'OffIce_A_shotAttempts', 'xGoalsForAfterShifts',
       'xGoalsAgainstAfterShifts', 'corsiForAfterShifts',
       'corsiAgainstAfterShifts', 'fenwickForAfterShifts',
       'fenwickAgainstAfterShifts'],
      dtype='object', length=154)

In [84]:
display(nhl_draft['overall_pick'].max())

293

In [158]:
active_skaters=skaters[skaters['season']==2022]
retired_skaters=skaters[skaters['name'].isin(active_skaters['name'])==False]
active_goalies=goalies[goalies['season']==2022]
retired_goalies=goalies[goalies['name'].isin(active_goalies['name'])==False]



In [174]:
retired=nhl_draft.dropna(subset=['to_year'])
retired=retired[retired['to_year']<2022]
retired['Mean Career Length']=retired['to_year']-retired['year']+1

rgp=retired.groupby('round').mean()
rgp=rgp[['Mean Career Length']]
seRetire=retired.groupby('round').sem()
rgp['Standard Error']=seRetire['Mean Career Length']
display(rgp)
dfi.export(rgp.style.format("{:.3f}"), 'meanNHLCareer.png')


,Mean Career Length,Standard Error
round,,
1,12.157647,0.244046
2,10.224181,0.227952
3,9.700658,0.253166
4,10.092664,0.271890
5,9.646154,0.319206
6,10.027174,0.335533
7,9.880240,0.340050
8,10.918919,0.426427
9,10.250000,0.506128


In [160]:
salaries=pd.read_csv('..\\Data\\nhl_salaries.csv')
salaries['Salary']=salaries['Salary'].replace('[\$,]', '', regex=True).astype(float)

pick=nhl_draft[['player', 'round', 'overall_pick']]
pick=pick.rename(columns={'player':'name'})
salaries=salaries.rename(columns={'Player':'name'})
skaterSalaries=skaters.merge(salaries, on='name', how='left')
skaterSalaries=skaterSalaries.merge(pick, on='name', how='left')
skaterSalaries=skaterSalaries[['name', 'season', 'round', 'Salary', 'overall_pick']]
goalieSalaries=goalies.merge(salaries, on='name', how='left')
goalieSalaries=goalieSalaries.merge(pick, on='name', how='left')
goalieSalaries=goalieSalaries[['name', 'season', 'round', 'Salary', 'overall_pick']]



In [161]:
goalieByRound=goalieSalaries.groupby('round').mean()
skaterByRound=skaterSalaries.groupby('round').mean()

skaterByRound.index.names = ['Round']
skaterByRound=skaterByRound.rename(columns={'Salary':'Mean Skater Salary'})
skaterByRound.index=skaterByRound.index.astype(int)
seSkater=skaterSalaries.groupby('round').sem()
skaterByRound['Standard Error']=seSkater['Salary']
dfi.export(skaterByRound[['Mean Skater Salary', 'Standard Error']].style.format("{:.0f}"), 'meanSkaterSalary.png')


In [162]:
goalieByRound.index.names = ['Round']
goalieByRound=goalieByRound.rename(columns={'Salary':'Mean Goalie Salary'})
goalieByRound.index=goalieByRound.index.astype(int)

seGoalie=goalieSalaries.groupby('round').sem()
goalieByRound['Standard Error']=seGoalie['Salary']
dfi.export(goalieByRound[['Mean Goalie Salary', 'Standard Error']].style.format("{:.0f}"), 'meanGoalieSalary.png')